In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple
plt.style.use('ggplot')

In [2]:
class DoesNotConvergeError(Exception):
    def __init__(self, result):
        self.result = result
        super(Exception, self).__init__("{}".format(result))

In [111]:
BisectionResult = namedtuple('BisectionResult', ['result', 'possible_error', 'res','iterations', 'iteration_list'])
def bisection(f, a, b, max_iterations=200, tol=1e-12):
    """
    Given a continuous function f(x) which crosses the x axis in the interval between a and b,
    this function searches for the intersection by iteratively halving the search space.
    f(a) and f(b) must have opposing signs.
    
    Returns a named tuple containing the resulting x coordinate, the maximum possible error,
    the residual, the number of iterations done, and the list of intermediate results for each iteration.
    """
    if f(a)*f(b) > 0: raise ValueError("f(a) and f(b) must not have the same sign")
    iterations = 0
    c = np.nan
    iteration_list = []
    while ((b - a) / 2) > tol and iterations < max_iterations:
        iterations+=1
        c = (a+b)/2
        iteration_list.append(c)
        if f(c)*f(a) > 0:
            a=c
        elif f(c)*f(b) > 0:
            b=c
        else: break
    possible_error=(b-a)/2
    return BisectionResult(c, possible_error, f(c),iterations, np.array(iteration_list))

In [4]:
#f = lambda k: (10+1/(2*k))*np.e**(-k)-7-1/(2*k)
f = lambda k: -(10+1/(2*k))*np.e**(-k)+7+1/(2*k)

In [112]:
xs = np.linspace(-3,3,1000)
ys = f(xs)

bisect_result = bisection(f, 0.1,1, max_iterations=10)
print(bisect_result)

plt.plot((bisect_result.result,bisect_result.result), (-200, 100), label="Bisection Result")
plt.plot(xs,ys, label="f(k)")

plt.title("Bisection")
plt.xlabel("k")
plt.ylabel("f(k)")
plt.legend()
plt.show()

BisectionResult(result=0.29599609375, possible_error=0.0004394531249999889, res=-0.005111024075844295, iterations=10, iteration_list=array([ 0.55      ,  0.325     ,  0.2125    ,  0.26875   ,  0.296875  ,
        0.2828125 ,  0.28984375,  0.29335937,  0.29511719,  0.29599609]))


In [76]:
f_fix = lambda k: k/7*(10+1/(2*k))*np.e**(-k)-1/14

In [114]:
FixedPointResult = namedtuple('FixedPointResult', ['result', 'iterations', 'iteration_list'])
def fixed_point(g, start, tol=1e-12,max_iterations=100):
    """
    Given a function g(x), finds the point g(r) = r by iteratively applying g.
    
    Not guaranteed to converge.
    
    Returns a namedtuple containing the resulting r value, the number of iterations taken,
    and the list of all intermediate iterations.
    """
    previous = start
    current = g(previous)
    iterations = 0
    iteration_list = [previous, current]
    while np.abs(current - previous) > tol:  
        iterations += 1
        previous = current
        current=g(current)
        if iterations >= max_iterations or np.isnan(current):
            raise DoesNotConvergeError(FixedPointResult(current, iterations, np.array(iteration_list)))
        iteration_list.append(current)
    return FixedPointResult(current, iterations, np.array(iteration_list))

In [113]:
xs = np.linspace(-0.5,1.5,1000)
ys = f_fix(xs)

point = fixed_point(f_fix, 0.5, tol=1e-8)
print(point)

#plt.plot(xs,ys, label="f_fix(k)")
#plt.plot(xs,xs, label="k")

#plt.plot((point,point), (plt.ylim(0), plt.ylim(1)), label="fixed point")

#plt.title("Fixed-Point")
#plt.xlabel("k")
#plt.ylabel("f(x)")
#plt.xlim(0,0.5)
#plt.legend()
#plt.show()

FixedPointResult(result=0.2967030689533232, iterations=42, iteration_list=array([ 0.40513123,  0.36217449,  0.33848537,  0.32418784,  0.31511516,
        0.30918043,  0.30522255,  0.3025494 ,  0.30072861,  0.29948129,
        0.29862348,  0.29803197,  0.29762334,  0.29734069,  0.29714501,
        0.29700946,  0.29691552,  0.29685039,  0.29680524,  0.29677393,
        0.29675221,  0.29673715,  0.29672671,  0.29671946,  0.29671443,
        0.29671095,  0.29670853,  0.29670685,  0.29670569,  0.29670488,
        0.29670432,  0.29670393,  0.29670366,  0.29670348,  0.29670335,
        0.29670326,  0.29670319,  0.29670315,  0.29670312,  0.2967031 ,
        0.29670309,  0.29670308,  0.29670307]))


In [115]:
def derive(f, h=1e-12):
    """Given a function f(x), returns a new function f'(x) which
    approximates the derivative of f at x"""
    def derived(x):
        return (f(x+h)-f(x))/h
    return derived

NewtonResult = namedtuple('NewtonResult', ['result', 'iterations', 'iteration_list', 'res'])
def newton(f, fprime, start, tol=1e-12):
    """Given a function f(x) and its derivative fprime(x), finds the
    point x where f(x)=0."""
    def g(x):
        return x - f(x)/fprime(x)
    fp_res = fixed_point(g, start, tol=tol)
    return NewtonResult(fp_res.result, fp_res.iterations, fp_res.iteration_list, f(fp_res.result))

def secant(f, start, tol=1e-12):
    """Given a function f(x), finds the point at which f(x)=0 using newton's
    method with an approximated derivative."""
    return newton(f, derive(f), start, tol=tol)

In [116]:
# Task 6
k=0.29670305
time_of_death = lambda t: 22 -37 + 0.5*t-1/(2*k) + (10 + 1/(2*k))*np.e**(-k*t)
derivative_of_death = lambda t: 0.5 + -k*(10 + 1/(2*k))*np.e**(-k*t)

result = newton(time_of_death, derivative_of_death, 0, tol=1e-8)
print(result)
result = secant(time_of_death, 1)
print(result)

NewtonResult(result=-1.3324871338230977, iterations=4, iteration_list=array([ 0.        , -1.68518659, -1.35202321, -1.33254968, -1.33248713,
       -1.33248713]), res=0.0)
NewtonResult(result=-1.3324871338230977, iterations=7, iteration_list=array([ 1.        , -2.61174049, -1.56237141, -1.34112883, -1.33250545,
       -1.33248713, -1.33248713, -1.33248713, -1.33248713]), res=0.0)


In [83]:
x = np.linspace(-10,5,1000)
plt.plot(x, time_of_death(x))
plt.show()
#plt.plot(x, derivative_of_death(x))
#time_of_death(-1.3324)

In [12]:
fixed_point(lambda t: time_of_death(t)+t, 1)

DoesNotConvergeError: FixedPointResult(result=5.579265880707039e+18, iterations=100, iteration_list=array([  5.39549713e+01,   6.42472717e+01,   7.96857210e+01,
         1.02843395e+02,   1.37579906e+02,   1.89684672e+02,
         2.67841822e+02,   3.85077546e+02,   5.60931132e+02,
         8.24711511e+02,   1.22038208e+03,   1.81388793e+03,
         2.70414671e+03,   4.03953488e+03,   6.04261714e+03,
         9.04724052e+03,   1.35541756e+04,   2.03145782e+04,
         3.04551821e+04,   4.56660880e+04,   6.84824468e+04,
         1.02706985e+05,   1.54043792e+05,   2.31049003e+05,
         3.46556820e+05,   5.19818545e+05,   7.79711132e+05,
         1.16955001e+06,   1.75430833e+06,   2.63144582e+06,
         3.94715204e+06,   5.92071137e+06,   8.88105037e+06,
         1.33215589e+07,   1.99823216e+07,   2.99734658e+07,
         4.49601819e+07,   6.74402562e+07,   1.01160368e+08,
         1.51740535e+08,   2.27610786e+08,   3.41416162e+08,
         5.12124226e+08,   7.68186322e+08,   1

In [117]:
def plot_convergence(result):
    errors = np.abs(result.result - result.iteration_list)
    changes = np.array([errors[i]/errors[i-1] for i in range(1, len(errors))])
    x = range(0, len(changes))
    plt.scatter(x, changes)
    print(sum(changes)/len(changes))
    print(changes)
    plt.xlabel("iterations")
    plt.ylabel("$e_n / e_{n-1}$")
    #plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
    plt.show()
    
def plot_quadratic_convergence(result):
    errors = np.abs(result.result - result.iteration_list)
    changes = np.array([errors[i]/(errors[i-1]**2) for i in range(1, len(errors))])
    x = range(0, len(changes))
    plt.scatter(x, changes)
    print(changes)
    plt.xlabel("iterations (N)")
    plt.ylabel("$e_n / e_{n-1}^2$")
    #plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
    plt.show()

In [118]:
result = bisection(time_of_death, -2, -1, max_iterations=26)
print(result)
plt.ylim(-1,2)
plot_convergence(result)
plt.show()

BisectionResult(result=-1.3324871212244034, possible_error=7.450580596923828e-09, res=-5.856152895944433e-08, iterations=26, iteration_list=array([-1.5       , -1.25      , -1.375     , -1.3125    , -1.34375   ,
       -1.328125  , -1.3359375 , -1.33203125, -1.33398438, -1.33300781,
       -1.33251953, -1.33227539, -1.33239746, -1.3324585 , -1.33248901,
       -1.33247375, -1.33248138, -1.3324852 , -1.33248711, -1.33248806,
       -1.33248758, -1.33248734, -1.33248723, -1.33248717, -1.33248714,
       -1.33248712]))
3.49709964307
[  4.92422564e-01   5.15388077e-01   4.70142738e-01   5.63506803e-01
   3.87300735e-01   7.90986449e-01   1.32122081e-01   3.28437878e+00
   3.47764209e-01   6.22442263e-02   6.53287356e+00   4.23464002e-01
   3.19262091e-01   6.61114003e-02   7.06299213e+00   4.29208473e-01
   3.35064935e-01   7.75193798e-03   6.30000000e+01   4.92063492e-01
   4.83870968e-01   4.66666667e-01   4.28571429e-01   3.33333333e-01
   0.00000000e+00]


In [130]:
# Fixed point
# Just adding +t to both sides doesn't converge, but this does
def best_fixed_g(t):
    return np.log(1+(5-0.5*t)/(10+(1/(2*k))))/(-k)

result = fixed_point(best_fixed_g, 40)
print(result)
plt.ylim(0,0.5)
plot_convergence(result)

FixedPointResult(result=nan, iterations=0, iteration_list=array([ 40.,  nan]))
nan
[ nan]


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: invalid value encountered in log


In [122]:
# Newton
result = newton(time_of_death, derivative_of_death, 0)
print(time_of_death(result.result))
print(result)
#plt.ylim(0,0.5)
plot_convergence(result)
plt.ylim(0,1)
plot_quadratic_convergence(result)

0.0
NewtonResult(result=-1.3324871338230977, iterations=5, iteration_list=array([ 0.        , -1.68518659, -1.35202321, -1.33254968, -1.33248713,
       -1.33248713, -1.33248713]), res=0.0)
nan

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in double_scalars



[  2.64692576e-01   5.53901458e-02   3.20177700e-03   1.02774878e-05
   0.00000000e+00              nan]
[ 0.1986455   0.15704631  0.1638905   0.16430796  0.                 nan]


In [50]:
def plot_iterations(expected, tol, x, f, **kwargs):
    y = []
    for start in x:
        try:
            result = f(start)
            if np.abs(result.result - expected) < tol:
                y.append(result.iterations)
            else:
                y.append(np.nan)
        except ValueError as e:
            y.append(np.nan)
        except DoesNotConvergeError as e:
            y.append(np.nan)
    plt.scatter(x,y, **kwargs)

In [128]:
# Comparing the convergence rate of the different methods. 
x = np.linspace(-100, 100, 10000)
expected = -1.332487133
tol = 0.0001
#plt.plot(x, time_of_death(x))
#plt.plot(x, derivative_of_death(x))
plt.xlabel("starting guess (x)")
plt.ylabel("number of iterations")
plt.xlim(x[0], x[-1])
plot_iterations(expected, tol, x, lambda start: fixed_point(best_fixed_g, start, tol=1e-8),color='r', label='fixed point')
plot_iterations(expected, tol, x, lambda start: newton(time_of_death, derivative_of_death, start, tol=1e-8),color='b', label='newton-raphson')
plt.legend()
plt.show()
x = np.linspace(0, 5, 1000)
plt.xlabel("size of initial interval")
plt.ylabel("number of iterations")
plot_iterations(expected, tol, x, lambda start: bisection(time_of_death, expected-start/2, expected+start/2),color='g')
plt.show()

# Analysis: Fixed point converges for all (?) negative values, but stops converging at just over x=30
# Newton's method converges for values under ~6, but is slower than fixed point for values under -30ish.
# near the correct answer, newton is significantly faster
# Bisection plot shows nothing interesting, since we already know how many iterations valid inputs will require to get a given tolerance

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: invalid value encountered in log
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: overflow encountered in double_scalars
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: overflow encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars


In [44]:
x = np.linspace(-100, 100, 1000)
expected = 0.296
tol = 0.01
#plt.plot(x, time_of_death(x))
#plt.plot(x, derivative_of_death(x))
plt.xlabel("starting guess (x)")
plt.ylabel("number of iterations")
plot_iterations(expected, tol, x, lambda start: fixed_point(f_fix, start, tol=1e-8),color='r')
#plot_iterations(expected, tol, x, lambda start: newton(time_of_death, derivative_of_death, start),color='b')
plt.show()


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
from scipy.optimize import fsolve

result, infodict, ier, mesg = fsolve(time_of_death, 3, full_output=True)
print(result)
print(infodict)
#plt.ylim(0,0.5)
#errors = np.abs(result.result - result.iteration_list)
#changes = np.array([errors[i]/errors[i-1] for i in range(1, len(errors))])
#x = range(0, len(changes))
#plt.scatter(x, changes)
#print(changes)
#plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
#plt.show()

In [ ]:
axvline